# train using prism

In [27]:
%load_ext autoreload
%autoreload 2

from load import load_3D
from liftpose.vision_3d import XY_coord, Z_coord
import yaml
import logging
from imp import reload
import torch
import numpy as np
import matplotlib.pyplot as plt
reload(logging)
logger = logging.getLogger(__name__).setLevel(logging.INFO)

# decleare data parameters
par_train = {'data_dir'     : "/data/LiftPose3D/fly_ventral_highres/network", # change the path 
             "template_dir" : '/home/user/Desktop/LiftPose3D/examples/fly_prism/out/',
           'out_dir'        : './out',
            "train_subjects": ["001", "002", "003"],
           'test_subjects'  : ["004"],
           'actions'        : ['PR']}

# merge with training parameters
par_data = yaml.full_load(open('param.yaml', "rb"))
par = {**par_data["data"], **par_train}

# load data
train, train_keypts, _ = load_3D(par["data_dir"], subjects=par['train_subjects'], actions=par['actions'] )
test,  test_keypts, _ = load_3D(par["data_dir"], subjects=par['test_subjects'],  actions=par['actions'] )
#project data to ventral view
train_2d, train_3d = XY_coord(train), Z_coord(train)
test_2d, test_3d  = XY_coord(test), Z_coord(test)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
from liftpose.main import train as lp3d_train
lp3d_train(train_2d=train_2d, test_2d=test_2d, train_3d=train_3d, test_3d=test_3d, 
           train_keypts=train_keypts, test_keypts=test_keypts, 
           **{k:par[k] for k in ['roots', 'target_sets', 'out_dir']})

[main.py:79]:INFO:Creating directory /home/user/Desktop/LiftPose3D/examples/fly_ventral_highres/out
[main.py:89]:INFO:Saving pre-processed 2D data at /home/user/Desktop/LiftPose3D/examples/fly_ventral_highres/out/stat_2d.pth.tar.
[main.py:111]:INFO:Saving pre-processed 3D data at /home/user/Desktop/LiftPose3D/examples/fly_ventral_highres/out/stat_3d.pth.tar.
[main.py:138]:INFO:Starting training model
[main.py:147]:DEBUG:
==================Options=================
[main.py:148]:DEBUG:{   'batch_size': 64,
    'data_dir': '/home/user/Desktop/LiftPose3D/examples/fly_ventral_highres/out',
    'dropout': 0.5,
    'epochs': 10,
    'exp': '',
    'is_train': True,
    'job': 8,
    'linear_size': 1024,
    'load': None,
    'lr': 0.001,
    'lr_decay': 100000,
    'lr_gamma': 0.96,
    'max_norm': True,
    'noise': None,
    'num_stage': 2,
    'out': '/home/user/Desktop/LiftPose3D/examples/fly_ventral_highres/out',
    'out_dir': '/home/user/Desktop/LiftPose3D/examples/fly_ventral_highres/

Epoch 0 | Loss Test 0 | Loss Train 1.3467|: 100%|██████████| 161/161 [00:02<00:00, 66.58it/s]
Epoch 1 | Loss Test 1.0519 | Loss Train 0.40511|: 100%|██████████| 161/161 [00:02<00:00, 66.77it/s]
Epoch 2 | Loss Test 0.95733 | Loss Train 0.139|: 100%|██████████| 161/161 [00:01<00:00, 82.26it/s]  
Epoch 3 | Loss Test 0.75994 | Loss Train 0.061662|: 100%|██████████| 161/161 [00:02<00:00, 79.81it/s] 
Epoch 4 | Loss Test 0.74613 | Loss Train 0.037966|: 100%|██████████| 161/161 [00:02<00:00, 70.46it/s]
Epoch 5 | Loss Test 0.71244 | Loss Train 0.031539|: 100%|██████████| 161/161 [00:02<00:00, 69.71it/s]
Epoch 6 | Loss Test 0.70805 | Loss Train 0.027766|: 100%|██████████| 161/161 [00:01<00:00, 88.53it/s]
Epoch 7 | Loss Test 0.68022 | Loss Train 0.026031|: 100%|██████████| 161/161 [00:02<00:00, 79.82it/s]
Epoch 8 | Loss Test 0.66295 | Loss Train 0.024374|: 100%|██████████| 161/161 [00:02<00:00, 67.85it/s]
Epoch 9 | Loss Test 0.67225 | Loss Train 0.022938|: 100%|██████████| 161/161 [00:02<00:00, 7

In [34]:
import torch
from liftpose.preprocess import *
from liftpose.main import flatten_dict
def create_xy_data( par ):
    """
    Creates 2d poses by projecting 3d poses with the corresponding camera
    parameters.
    """
    # Load data
    test_set, _, _ = load_3D(par['data_dir'], par, subjects=par['test_subjects'], actions=par['actions'] )
    test_set = flatten_dict(test_set)

    # anchor points
    test_set, _ = anchor_to_root( test_set, par['roots'], par['target_sets'], dim=2)    

    # Compute normalization statistics
    mean = torch.load(par['template_dir'] + 'stat_2d.pth.tar')['mean']
    std = torch.load(par['template_dir'] + 'stat_2d.pth.tar')['std']

    refs = np.array(par['template_coords']).astype(int)
    refs = np.sort(np.hstack((refs*2, refs*2+1)))
    
    # Divide every dimension independently
    test_set = normalize(test_set, mean[refs], std[refs] )

    #select coordinates to be predicted and return them as 'targets_3d'
    test_set, targets_2d = remove_roots(test_set, par['target_sets'], 2)
    _, targets_1d = remove_roots(test_set.copy(), par['target_sets'], 1)

    return test_set, mean, std, targets_1d, targets_2d


par = {'data_dir': '/data/LiftPose3D/fly_ventral_lowres/102906_s1a5_p6-0/',
'template_dir': '/home/user/Desktop/LiftPose3D/examples/fly_prism/out/',
'out_dir':'./out/',
'in_dim': 2,
'out_dim': 1,
'test_subjects': [0],
'actions': ['all'], #actions to load
'target_sets': [[ 1,  2,  3],  [5,  6,  7], [9, 10, 11],
               [13, 14, 15], [17, 18, 19], [21, 22, 23]],
'roots': [0, 4, 8, 12, 16, 20],
'template_coords': [ 1,2,3,4,   6,7,8,9,  11,12,13,14, #coordinates in the prism system
                  16,17,18,19,  21,22,23,24,  26,27,28,29]}

test_set, mean, std, targets_1d, targets_2d = \
create_xy_data(par)

torch.save(test_set, par['out_dir'] + '/test_2d.pth.tar')
torch.save({'mean': mean, 'std': std, 
            'targets_2d': targets_1d, 'targets_3d': targets_2d,
            'input_size': len(targets_2d),
            'output_size': len(targets_1d)},
            par['out_dir'] + '/stat_2d.pth.tar')

In [39]:
from liftpose.main import test as lp3d_test
lp3d_test(par['out_dir'])

[main.py:154]:INFO:starting testing in path: ./out/
[lift.py:54]:INFO:total params: 4.28M
[lift.py:60]:INFO:loading ckpt from './out//ckpt_best.pth.tar'
[lift.py:68]:INFO:ckpt loaded (epoch: 8 | err: 2.041229486465454)


RuntimeError: size mismatch, m1: [64 x 36], m2: [48 x 1024] at /opt/conda/conda-bld/pytorch_1591914895884/work/aten/src/THC/generic/THCTensorMathBlas.cu:283